In [31]:
%load_ext autoreload
%autoreload 2

from __future__ import annotations

# Add parent directory to path.
from pathlib import Path
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # https://stackoverflow.com/a/64438413
fdir = Path(os.path.abspath('')).resolve() # Directory of current file.
path = fdir/'..'
if path not in sys.path:
    sys.path.append(str(path))

# Complete imports.
import makassar_ml as ml
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds
import seaborn as sns
sns.set() # Use seaborn themes.

# List all GPUs visible to TensorFlow.
gpus = tf.config.list_physical_devices('GPU')
print(f"Num GPUs Available: {len(gpus)}")
for gpu in gpus:
    print(f"Name: {gpu.name}, Type: {gpu.device_type}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Num GPUs Available: 1
Name: /physical_device:GPU:0, Type: GPU


In [32]:
keras.backend.clear_session()

In [33]:
# Configure root paths.
config = dict(
    roots=dict(
        hp_tuning_root=Path('~/research/makassar/hp_tuning').expanduser(),
        image_root=Path('~/research/makassar/images').expanduser(),
    ),
)

In [34]:
epochs = 30
batch_size = 256

In [35]:
in_feat = ['TEMP','DEWP','PRES','Iws']
out_feat = ['pm2.5','Ir']
in_seq_len = 24 # 1 day
out_seq_len = 1 # 1 hour

ds_train, ds_val, ds_test = ml.datasets.beijingpm25.load_data(
    path=Path('~/research/makassar/datasets/beijing_pm25').expanduser(),
    in_feat=in_feat,
    out_feat=out_feat,
    in_seq_len=in_seq_len,
    out_seq_len=out_seq_len,
    shift=1,
    split=[0.7,0.2,0.1],
    shuffle=False,
    batch_size=batch_size,
)

ds_train = ds_train.prefetch(tf.data.AUTOTUNE).cache()
ds_val = ds_val.prefetch(tf.data.AUTOTUNE).cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE).cache()

In [36]:
for seq_in, seq_out in ds_train:
    print(seq_in.shape)
    print(seq_out.shape)
    break

(256, 24, 4)
(256, 1, 2)


In [39]:
model = ml.models.lstm_net.build_model(
    in_seq_len=in_seq_len,
    in_feat=len(in_feat),
    out_feat=len(out_feat),
    lstm_units=[128, 64],
    fc_units=[256, 256],
    dropout=0.1,
)


optimizer = keras.optimizers.Adam(learning_rate=1e-3)
metrics = ['mae']
model.compile(
    optimizer=optimizer,
    loss='mse',
    metrics=metrics,
)
model.summary()

AttributeError: 'CacheDataset' object has no attribute 'shape'

In [ ]:
model, history, metrics = ml.training.train_evaluate_model(
    model,
    datagen_train=ds_train,
    datagen_val=ds_val,
    datagen_test=ds_test,
    epochs=epochs,
    checkpoint_path=config['roots']['hp_tuning_root']/'lstm'/'lstm.h5',
    history_path=config['roots']['hp_tuning_root']/'lstm'/'lstm_history.csv',
    metrics_path=config['roots']['hp_tuning_root']/'lstm'/'lstm_metrics.json',
)

Epoch 1/30
103/297 [=========>....................] - ETA: 2:23 - loss: 2.3223 - accuracy: 0.4078

In [ ]:
# Train.
# strategy = tf.distribute.get_strategy()
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=epochs,
    verbose=1,
)

Epoch 1/30
297/297 [==============================] - 294s 807ms/step - loss: 1.6595 - accuracy: 0.5582 - val_loss: 815.6634 - val_accuracy: 0.1016
Epoch 2/30
297/297 [==============================] - 115s 387ms/step - loss: 0.4061 - accuracy: 0.8688 - val_loss: 0.5723 - val_accuracy: 0.8353
Epoch 3/30
297/297 [==============================] - 115s 387ms/step - loss: 0.2160 - accuracy: 0.9306 - val_loss: 0.2851 - val_accuracy: 0.9081
Epoch 4/30
297/297 [==============================] - 115s 387ms/step - loss: 0.1570 - accuracy: 0.9486 - val_loss: 3.9073 - val_accuracy: 0.4665
Epoch 5/30
297/297 [==============================] - 115s 387ms/step - loss: 0.1432 - accuracy: 0.9515 - val_loss: 0.2236 - val_accuracy: 0.9321
Epoch 6/30
297/297 [==============================] - 115s 387ms/step - loss: 0.0880 - accuracy: 0.9707 - val_loss: 0.5948 - val_accuracy: 0.8498
Epoch 7/30
297/297 [==============================] - 115s 387ms/step - loss: 0.0832 - accuracy: 0.9715 - val_loss: 0.6520

In [ ]:
# Evaluate the newly trained model.
test_metrics = model.evaluate(ds_test)

43/43 [==============================] - 51s 1s/step - loss: 3523.6990 - accuracy: 0.0120


In [ ]:
import json

# Create dictionary of metrics to return and preserve in file.
metrics = {}
for i, (key, val) in enumerate(history.history.items()):
    metrics[key] = val[-1]
    if not key.startswith('val_'):
        metrics[f"test_{key}"] = test_metrics[i]

# Dump metrics to JSON file.
with open('lstm_metrics.json', 'w') as f:
    json.dump(metrics, f)

In [ ]:
# Plot train/val performance for best model.
allmetrics = ['loss'] + metrics
fig, ax = plt.subplots(nrows=len(allmetrics), ncols=1, figsize=(10,len(allmetrics)*3), sharex=True, constrained_layout=True)
for j, key in enumerate(sorted(allmetrics)):
    ax[j].plot(history[key], label=f'train', linestyle='-')
    ax[j].plot(history[f"val_{key}"], label=f'val', linestyle='--')
    ax[j].set_xlim(0, len(history[key])-1)
    if j == len(allmetrics)-1:
        ax[j].set_xlabel('epoch')
    ax[j].set_ylabel(key)
    # ax[j].legend(loc='upper left')

handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncol=2, bbox_to_anchor=(0.5,0.0))
path = Path(config['roots']['image_root'])/f"lstm.png"
fig.savefig(path, bbox_inches='tight')
print(path)